<a href="https://colab.research.google.com/github/R4HUL-ROY/Multimodal_feature_extraction/blob/main/Tobaco3482_SMALLBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import pathlib
import tensorflow_hub as hub
!pip install tensorflow_text
import tensorflow_text as text

     |████████████████████████████████| 4.9 MB 5.3 MB/s 
     |████████████████████████████████| 462 kB 34.1 MB/s 


In [2]:
data_root = pathlib.Path('/content/drive/MyDrive/tobaco_OCR/')

print(data_root)
for item in data_root.iterdir():
  print(item)

/content/drive/MyDrive/tobaco_OCR
/content/drive/MyDrive/tobaco_OCR/Resume
/content/drive/MyDrive/tobaco_OCR/Memo
/content/drive/MyDrive/tobaco_OCR/Letter
/content/drive/MyDrive/tobaco_OCR/Report
/content/drive/MyDrive/tobaco_OCR/Email
/content/drive/MyDrive/tobaco_OCR/ADVE
/content/drive/MyDrive/tobaco_OCR/News
/content/drive/MyDrive/tobaco_OCR/Note
/content/drive/MyDrive/tobaco_OCR/Form
/content/drive/MyDrive/tobaco_OCR/Scientific


In [3]:
def get_file_paths_and_labels(data_root):
     text_paths = [str(path) for path in data_root.glob('*/*.txt')]
     labels = [p.split("/")[-2] for p in text_paths]
     return text_paths, labels

text_paths, labels = get_file_paths_and_labels(data_root)
print(text_paths)
print(labels)
print(len(text_paths))
print(len(labels))

['/content/drive/MyDrive/tobaco_OCR/Resume/50521422-1423.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50630631-0632.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/40028776-8777.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/10150247_10150256.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50538795-8796.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50201456-1467.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50719747-9748.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50264605-4625.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50590985-0986.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50296092-6093.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50535699-5699.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50239379-9380.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50467009-7010.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50425154-5155.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/40010133-0134.txt', '/content/drive/MyDrive/tobaco_OCR/Resume/50439450-9451.txt', '/c

In [4]:
df = pd.DataFrame(list(zip(text_paths, labels)),
               columns =['text_path', 'data_label'])
df.head()

,text_path,data_label
0,/content/drive/MyDrive/tobaco_OCR/Resume/50521...,Resume
1,/content/drive/MyDrive/tobaco_OCR/Resume/50630...,Resume
2,/content/drive/MyDrive/tobaco_OCR/Resume/40028...,Resume
3,/content/drive/MyDrive/tobaco_OCR/Resume/10150...,Resume
4,/content/drive/MyDrive/tobaco_OCR/Resume/50538...,Resume


In [5]:
import re
def preprocess_text(text_string):
    preprocessed_string = re.sub(r'[^\w\s]','',text_string)
    preprocessed_string = preprocessed_string.replace('\n',' ')
    preprocessed_string = re.sub(' +', ' ', preprocessed_string)
    return preprocessed_string


In [6]:
def get_text_from_path(path):
    with open(path) as f:
        lines = f.readlines()
        lines  = ' '.join(lines)
        f.close()
    return lines

out_text = get_text_from_path('/content/drive/MyDrive/tobaco_OCR/ADVE/0000435350.txt') 
# out_text = preprocess_text(out_text)
print(out_text)   



 TE che fitm
 m66400 7127
 KOOLS are the only cigarettes that taste
 good when you have &® cold. They taste even
 better when you don't.
 Job No, K-2978
 ‘Mevapapars—300 iner—Mareh & April, 1956
 (5 9-4 in 4 108 ines) Pinel Proof (7) March 15, 1956



In [7]:
## Tokenize, Lemmatize, stopwords removal
import spacy 
import nltk
nlp = spacy.load("en", disable=['parser', 'tagger', 'ner'])
from nltk.corpus import stopwords
nltk.download('stopwords')
stops = stopwords.words("english")

def normalize(comment, lowercase, remove_stopwords):
    if lowercase:
        comment = comment.lower()
    comment = nlp(comment)
    lemmatized = list()
    for word in comment:
        lemma = word.lemma_.strip()
        if lemma:
            if not remove_stopwords or (remove_stopwords and lemma not in stops):
                lemmatized.append(lemma)
    return " ".join(lemmatized)

normalize("counting playing the Home", lowercase=True, remove_stopwords=True)    

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


'count play home'

In [8]:
# preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
# encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-12_H-128_A-2/2"
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

In [11]:
texts = [get_text_from_path(this_path) for this_path in text_paths]

In [12]:
# texts = [preprocess_text(this_text) for this_text in texts]

In [13]:
# texts = [normalize(this_text, lowercase=True, remove_stopwords=True) for this_text in texts]

In [14]:
text_preprocessed = bert_preprocess_model(texts)

In [15]:
bert_results = bert_model(text_preprocessed)

In [16]:
# out = bert_results['sequence_output']
# r1, c1, c2 = out.shape
# out = np.array(out)
# out = out.reshape(r1, c1*c2)
# out.shape

In [17]:
out = bert_results['pooled_output']
out = np.array(out)
out.shape

(3482, 128)

In [18]:
vec_df = pd.DataFrame(out)

In [19]:
vec_df['data_label'] = df['data_label']

In [20]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
vec_df['data_label']= le.fit_transform(vec_df['data_label'])

In [21]:
vec_df['data_label'].value_counts()

4    620
1    599
3    567
2    431
7    265
9    261
0    230
6    201
5    188
8    120
Name: data_label, dtype: int64

In [22]:
vec_df.head()

,0,1,2,3,4,5,6,7,8,9,...,119,120,121,122,123,124,125,126,127,data_label
0,0.764275,-0.160210,-0.399372,0.974774,-0.119610,-0.721413,0.144863,-0.040624,-0.841448,0.529938,...,-0.987908,0.079127,0.807886,-0.996122,-0.180103,-0.990588,-0.932363,0.604320,-0.925260,8
1,0.710225,-0.551844,-0.294849,0.991579,0.559680,-0.201199,0.129197,-0.015183,-0.879636,0.357707,...,-0.990777,0.059642,0.746588,-0.998732,-0.070934,-0.995571,-0.953123,0.354050,-0.933002,8
2,0.543321,-0.267175,-0.782879,0.984169,0.586311,0.196906,0.030937,0.015753,-0.440458,0.625234,...,-0.988861,0.375319,0.702832,-0.997452,-0.031873,-0.990824,-0.875941,0.174660,-0.624428,8
3,0.671931,-0.652462,-0.527962,0.987529,0.429928,-0.375012,0.113949,0.120384,-0.761070,0.338829,...,-0.987223,0.500340,0.838875,-0.997991,-0.016040,-0.986776,-0.952415,0.628187,-0.922828,8
4,0.837430,-0.586352,-0.544527,0.987718,0.738552,-0.303306,0.116027,0.049022,-0.814585,0.431844,...,-0.987282,0.132122,0.826923,-0.994680,-0.029496,-0.987402,-0.962971,-0.088003,-0.969998,8


In [23]:
X =vec_df.iloc[:,:-1]
y =vec_df['data_label']

print(X.shape)
print(y.shape)

(3482, 128)
(3482,)


In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

In [25]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [26]:
from sklearn.ensemble import RandomForestClassifier
model=RandomForestClassifier(n_estimators=100)

In [27]:
model.fit(X_train_scaled,y_train)
y_pred=model.predict(X_test_scaled)

In [28]:
from sklearn import metrics
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.6277511961722488


In [29]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[ 22,   4,  10,   2,   2,   3,  10,   2,   1,   0],
       [  0, 157,   6,   4,  12,   0,   0,   1,   0,   0],
       [  2,   8,  96,   2,  13,   0,   1,   1,   0,   2],
       [  0,   9,  13,  93,  43,   0,   1,  11,   1,   3],
       [  3,  13,  10,  38, 136,   2,   1,   3,   0,   4],
       [  4,   2,   3,  18,  10,  16,   1,   2,   1,   0],
       [  5,   7,   5,   2,   0,   0,  35,   0,   0,   1],
       [  0,   3,  11,  11,  13,   2,   0,  27,   0,   8],
       [  0,   0,   1,   1,   0,   0,   0,   0,  35,   0],
       [  0,   0,   6,  11,   8,   1,   2,   8,   1,  39]])

In [30]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.61      0.39      0.48        56
           1       0.77      0.87      0.82       180
           2       0.60      0.77      0.67       125
           3       0.51      0.53      0.52       174
           4       0.57      0.65      0.61       210
           5       0.67      0.28      0.40        57
           6       0.69      0.64      0.66        55
           7       0.49      0.36      0.42        75
           8       0.90      0.95      0.92        37
           9       0.68      0.51      0.59        76

    accuracy                           0.63      1045
   macro avg       0.65      0.60      0.61      1045
weighted avg       0.63      0.63      0.62      1045

